In [2]:
using GpABC, DifferentialEquations, Distributions, Distances, Plots

In [3]:
	#threshold_schedule = [20, 15, 10, 5, 3, 2.5, 2, 1.7, 1.5]
	threshold_schedule = [20, 15, 10]
	summary_statistic = "keep_all"
	max_iter = 1e4
	n_particles = 200
	distance_metric = euclidean

	#
	# Experimental data - from ABCSysBio example at
	# https://github.com/jamesscottbrown/abc-sysbio/blob/master/examples/SBML/Example1/SIR.ipynb
	#
	times = [0.0, 0.6, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]
	data = [[ 20.     ,  10.     ,   0.     ],
	       [  0.12313,  13.16813,   9.42344],
	       [  0.12102,   7.17251,  11.18957],
	       [  0.09898,   2.36466,  10.0365 ],
	       [  0.37887,   0.92019,   6.87117],
	       [  1.00661,   0.61958,   4.44955],
	       [  1.20135,   0.17449,   3.01271],
	       [  1.46433,   0.28039,   1.76431],
	       [  1.37789,   0.0985 ,   1.28868],
	       [  1.57073,   0.03343,   0.81813],
	       [  1.4647 ,   0.28544,   0.52111],
	       [  1.24719,   0.10138,   0.22746],
	       [  1.56065,   0.21671,   0.19627]]

	data = hcat(data...)

	#
	# Define priors and initial conditions for each model
	#

	#
	# Priors and initial conditions - these are model-specfic as each model can 
	# have different numbers of parameters/species
	#
	priors1 = [Uniform(0.0, 5.0) for i in 1:4]
	ic1 = [20.0, 10.0, 0.0]

	priors2 = vcat([Uniform(0.0, 5.0) for i in 1:4], Uniform(0.0, 10.0))
	ic2 = [20.0, 0.0, 10.0, 0.0]

	modelprior = DiscreteUniform(1, 2)

	# p = (alpha, gamma, d, v)
	# x = (S, I, R)
	function model1(dx, x, p, t)
	    dx[1] = p[1] - p[2]*x[1]*x[2] - p[3]*x[1] # dS/dt = alpha - gamma*S*I - d*S
	    dx[2] = p[3]*x[1]*x[2] - p[4]*x[2] - p[3]*x[2] # dI/dt = gamma*S*I - v*I - d*I
	    dx[3] = p[4]*x[2] - p[3]*x[3] # dR/dt = v*I - d*R
	end

	# p = (alpha, gamma, d, v, delta)
	# x = (S, L, I, R)
	function model2(dx, x, p, t)
	    dx[1] = p[1] - p[2]*x[1]*x[3] - p[3]*x[1] # dS/dt = alpha - gamma*S*I - d*S
	    dx[2] = p[2]*x[1]*x[3] - p[5]*x[2] - p[3]*x[2] # dL/dt = gamma*S*I - delta*L - d*L
	    dx[3] = p[5]*x[2] - p[4]*x[3] - p[3]*x[3] # dI/dt = delta*L - v*I - d*I
	    dx[4] = p[4]*x[3] - p[3]*x[4] # dR/dt = v*I - d*R
	end

	ics = [ic1, ic2]

	# Define simulator functions for each model

	simulator1(params) = Array{Float64,2}(
	    solve(ODEProblem(model1, ics[1], (times[1], times[end]), params), saveat=times, force_dtmin=true))

	# Model2 contains the species L, which is not measured - we remove it from the returned ODE solution
	# so that it can be compared to the reference data "data", which only contains S, I and R
	simulator2(params) = Array{Float64,2}(
	    solve(ODEProblem(model2, ics[2], (times[1], times[end]), params), saveat=times, force_dtmin=true))[[1,3,4],:]
	
	#
	# For tests on output shapes
	#
	n_params = [4,5]


	#
	# Do model selection using simulation
	#
	input = SimulatedModelSelectionInput(2, n_particles, threshold_schedule, modelprior,
	    [priors1, priors2], summary_statistic, distance_metric, [simulator1, simulator2], max_iter)

	ms_res = model_selection(input, data);
	
	# User-level function for the same computation
	ms_res  = model_selection(data, n_particles, threshold_schedule, [priors1, priors2],
		summary_statistic, [simulator1, simulator2])

Simulated model selection
Population 1
Completed 691 iterations, accepting 200 particles in total
Number of accepted parameters: Model 1: 87	Model 2: 113
Population 2
Completed 1087 iterations, accepting 200 particles
Number of accepted parameters: Model 1: 46	Model 2: 154
Population 3
Completed 2433 iterations, accepting 200 particles
Number of accepted parameters: Model 1: 40	Model 2: 160


LoadError: [91mMethodError: no method matching model_selection(::Array{Float64,2}, ::Int64, ::Array{Int64,1}, ::Array{Array{Distributions.Uniform{Float64},1},1}, ::String, ::Array{Function,1})[0m
Closest candidates are:
  model_selection(::AbstractArray{Float64,2}, ::Integer, [91m::AbstractArray{Float64,1}[39m, ::AbstractArray{AD<:AbstractArray{D<:Distributions.Distribution{Distributions.Univariate,Distributions.Continuous},1},1}, ::Union{AbstractArray{String,1}, Function, String}, ::AbstractArray{Function,1}; model_prior, distance_function, max_iter) where {D<:Distributions.Distribution{Distributions.Univariate,Distributions.Continuous}, AD<:AbstractArray{D<:Distributions.Distribution{Distributions.Univariate,Distributions.Continuous},1}} at /home/jonathan/Documents/GpAbc.jl/src/abc/simulation.jl:121[39m